___
<a href='https://cafe.naver.com/jmhonglab'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

# 지도학습(Supervised Learning) - 회귀(Regression)

입력에 대해서 기대하는 출력이 명확한 데이터셋을 가지고 있을 경우에는 지도학습을 사용할 수 있습니다.  
지도학습은 크게 두 가지 문제에 사용됩니다. 하나는 회귀(regression)이고 다른 하나는 분류(classification)입니다.  
회귀는 주어진 값에 대해서 연속적인 값을 출력하는 것이고 분류는 개와 고양이를 구분하듯이 입력으로 받은 데이터를 구분짓는 것을 의미합니다.  
여기서는 아주 간단한 직선 형태의 모델을 사용하는 **선형 회귀** 예제를 통해서 지도학습이 어떻게 이루어지는 지 알아보겠습니다.

### 공부 시간으로 점수 예상하기

아래와 같은 데이터를 가지고 있을 때 3시간을 공부하면 몇 점을 받을 것으로 예상할 수 있을까?  

입력(공부시간) -> 모델 -> 출력(점수)

출력인 점수가 연속적인 값이기 때문에 회귀(regression) 문제입니다.

|샘플|공부 시간|점수|
|:-:|:-:|:-:|
|학생1|1|15|
|학생2|5|55|
|학생3|8|60|

In [ ]:
import matplotlib.pyplot as plt

hours = [1.0, 5.0, 8.0] # 공부 시간
scores = [15.0, 55.0, 60.0]

plt.scatter(hours, scores, s = 100)

##### 선형 회귀(linear regression)

여러 개의 데이터 샘플을 모두 **적당히** 만족시킬 수 있는 직선을 어떻게 찾을 것인가?

훈련(train): 모델을 데이터에 대해 최적화
1. 초기 모델을 가정하고 난수(random numbers)로 초기화
1. 모델에 입력 데이터(공부 시간)을 넣어서 어떤 점수를 출력하는 지 본다.
1. 모델의 출력과 실제 데이터(점수)의 차이를 이용해서 얼마나 틀렸는지 손실(loss)을 계산한다.
1. 현재 모델에 대한 손실의 gradient를 계산해서 손실을 줄일 수 있는 방향을 찾는다.
1. 찾은 gradient를 이용해서 실제로 모델을 업데이트한다.
1. 손실이 충분히 작아질 때까지 또는 미리 지정해놓은 반복횟수만큼 2단계로 돌아가서 반복한다.

한 번에 여러 데이터 샘플에 대해서 gradient descent를 적용하고 싶다면 3단계와 4단계에서 여러 샘플들에 대한 loss의 평균을 사용합니다.


In [ ]:
import torch
import numpy as np
import copy # 모델 복사용

torch.manual_seed(0) # 디버깅할 때 같은 결과가 나오도록

# PyTorch모델 만들기
# 선형 모델 y = ax + b 가정
class LinearModel(torch.nn.Module): # Module 상속
    def __init__(self):
        super().__init__()

        self.a = torch.randn(1)
        self.b = torch.randn(1)

        self.a = torch.nn.Parameter(self.a) # Parameter로 등록
        self.b = torch.nn.Parameter(self.b) # Parameter로 등록
    
    def forward(self, x): # __call__() 처럼 사용됨
        y = self.a * x + self.b # y = ax + b
        return y

# 데이터 준비
x_input = torch.tensor([1, 5, 8], dtype=torch.float) # 공부시간, torch.Size([3])
y_target = torch.tensor([15, 55, 60], dtype=torch.float) # 점수, torch.Size([3])

# 모델 객체 만들기 (랜덤하게 초기화)
model = LinearModel()

criterion = torch.nn.MSELoss() # Mean Squared Error (MSE) loss
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2) # Stochastic Gradient Descent

num_epochs = 1000 # 데이터셋의 모든 샘플들에 대해 1번씩 훈련하는 것을 한 epoch라고 부릅니다.

loss_history = []
model_history = []

for epoch in range(1, num_epochs + 1): # 전체 데이터에 대해서 한 번 훈련시키는 것을 epoch

    optimizer.zero_grad() # 이전 훈련에서 계산했던 gradient 삭제

    y_pred = model(x_input) # torch.Size([3])

    # error = y_target - y_pred # torch.Size([3])
    # loss = (error ** 2).mean() # Mean Squared Error (MSE) loss, torch.Size([])
    loss = criterion(y_target, y_pred) # 손실(loss) 계산

    loss.backward() # 모델에 대한 loss의 gradient를 계산

    optimizer.step() # 모델의 parameters에 대해 gradient descent 수행

    loss_history.append(loss.item())
    model_history.append(copy.deepcopy(model)) # 용량이 클 경우 모델이나 일부 출력을 파일에 기록

    if epoch % (num_epochs // 10) == 0:
        print(f"Epoch {epoch}: loss = {loss.item()}")

In [ ]:
# 그래프로 결과 확인
import matplotlib.pyplot as plt
import math

def visualize(x_input, y_target, loss_history, model_history, marker_size = 1):

    x_input = x_input.numpy()
    y_target = y_target.numpy()

    plt.figure(figsize=(24, 6))

    plt.subplot(141)

    plt.scatter(x_input, y_target, s = marker_size) # 모든 데이터 샘플 그리기

    # 훈련 마지막 모델의 출력 그리기
    x = np.linspace(x_input.min(), x_input.max(), 10)
    y = model_history[-1](torch.tensor(x)).detach().numpy() 
    plt.plot(x, y, c="red")
    #[참고] tensor.detach().numpy(): 훈련에 필요한 정보들은 빼고 순수 값만 넘파이로 변환

    plt.subplot(142)

    num_steps = len(loss_history) // 10
    plt.scatter(x_input, y_target, s = marker_size) # 샘플 그리기
    colors = plt.cm.rainbow(np.linspace(0,1,10))
    for i in range(0, len(model_history), num_steps):
        y = model_history[i](torch.tensor(x)).detach().numpy()
        plt.plot(x, y, c=colors[i//num_steps])

    plt.subplot(143)

    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.plot(loss_history)

    plt.subplot(144)

    plt.xlabel("Epoch")
    plt.ylabel("Log loss")
    plt.plot([math.log(l) for l in loss_history])

    plt.show()

visualize(x_input, y_target, loss_history, model_history, marker_size = 100)

In [ ]:
trained_model = model_history[-1]

y_pred = trained_model(torch.tensor([3.0]))

print(y_pred.item()) # 32.268489837646484

##### 어떠한 손실 함수를 사용할 것인가?


[참고] loss는 분야에 따라 cost, energy 등 다른 이름으로 불리기도 합니다.


In [ ]:
import matplotlib.pyplot as plt

error = np.linspace(-1, 1, 100)
loss1 = error
loss2 = abs(error)
loss3 = 0.5 * error ** 2 # 앞에 0.5 붙이는 경우도 있고 붙이지 않는 경우도 있다.

plt.figure(figsize=(18, 6))

plt.subplot(131)
plt.xlabel("Error")
plt.ylabel("Error Error")
plt.plot(error, loss1)

plt.subplot(132)
plt.xlabel("Error")
plt.ylabel("Abs Error")
plt.plot(error, loss2)

plt.subplot(133)
plt.xlabel("Error")
plt.ylabel("Squared Error")
plt.plot(error, loss3)

### [실습] [다이아몬드](https://www.kaggle.com/datasets/shivam2503/diamonds?resource=download) 데이터셋 

2.5 캐럿 다이아몬드의 예상 가격은?

In [ ]:
# 데이터셋 읽어들이기

import pandas as pd

df = pd.read_csv('diamonds.csv')

df.plot.scatter(x="carat", y="price", s=0.1, figsize=(9, 4))


$x$는 carat(다이아몬드의 무게), $y$는 price(가격)에 대해 선형 회귀를 수행  
Learning rate와 반복 회수를 스스로 결정해서 여러가지로 테스트 해보기  
이때, learning rate가 너무 크면 수치적으로 불안정해져서 문제 발생    
반복 횟수가 너무 적으면 결과가 정확하지 않음  

In [ ]:
import torch
import numpy as np
import copy # 모델 복사용

torch.manual_seed(0) # 디버깅할 때 같은 결과가 나오도록


##### [실습] 2차 곡선 맞춤 (Curve Fitting)

모델을 $y = ax^2 + bx + c$로 바꿔보세요

In [ ]:
import torch
import numpy as np



##### [참고] [scikit-learn](https://scikit-learn.org/stable/)의 선형회귀

데이터 사이언스에서는 scikit-learn을 많이 사용합니다. scikit-learn에서 선형회귀 하는 코드이니 참고하세요.

In [ ]:
# sklearn
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

df = pd.read_csv('diamonds.csv')

x_input = df["carat"].to_numpy() # input
y_target = df["price"].to_numpy() # output

x_input = np.expand_dims(x_input, axis=1)
y_target = np.expand_dims(y_target, axis=1)

regr = LinearRegression()
regr.fit(x_input, y_target)

y_pred = regr.predict(x_input)

plt.figure(figsize=(8, 6))
plt.scatter(x_input.squeeze(), y_target.squeeze(), s = 0.1) # 샘플 그리기
plt.plot(x_input.squeeze(), y_pred.squeeze(), c="red")

print(regr.predict([[2.5]])) # [[17134.70346488]]